<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h2> Part 1: Getting the data from the Webseite into a Pandas DataFrame </h2>

### Get the website html text:

In [2]:
import requests
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text
html_content[:100]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title'

### Pharse it via BeautifulSoup 
(if required: pip install beautifulsoup4)

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())

"
      <a dir="ltr" href="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;oldid=979555370">
       https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;oldid=979555370
      </a>
      "
     </div>
    </div>
    <div class="catlinks" data-mw="interface" id="catlinks">
     <div class="mw-normal-catlinks" id="mw-normal-catlinks">
      <a href="/wiki/Help:Category" title="Help:Category">
       Categories
      </a>
      :
      <ul>
       <li>
        <a href="/wiki/Category:Communications_in_Ontario" title="Category:Communications in Ontario">
         Communications in Ontario
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Postal_codes_in_Canada" title="Category:Postal codes in Canada">
         Postal codes in Canada
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Toronto" title="Category:Toronto">
         Toronto
        </a>
       </li>
       <li>
        <a href="/wik

### Create a nested list out of the table.
Split on "," and remove new lines "\n" and spaces " "

In [55]:
data = {"PostalCode":[],"Borough":[],"Neighborhood":[]}
column = []
table = soup.find('table')
for row in table.find_all("tr"):
    for col in row.find_all("td"):
        txts = col.get_text().split(",")
        txts = [txt.rstrip("\n").lstrip(" ") for txt in txts]
        column.append(txts)
    if(column):
        data["PostalCode"].append(column[0][0])
        data["Borough"].append(column[1][0])
        data["Neighborhood"].append(column[2])
    column=[]

print(len(data["PostalCode"]))
assert len(data["PostalCode"]) == len(data["Borough"]) 
assert len(data["Neighborhood"]) == len(data["Borough"]) 

180


### Make the nested list a pandas dataframe

In [226]:
import pandas as pd

df_toronto = pd.DataFrame(data = data)
df_toronto


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,[Not assigned]
1,M2A,Not assigned,[Not assigned]
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
...,...,...,...
175,M5Z,Not assigned,[Not assigned]
176,M6Z,Not assigned,[Not assigned]
177,M7Z,Not assigned,[Not assigned]
178,M8Z,Etobicoke,"[Mimico NW, The Queensway West, South of Bloor..."


Final output:

In [227]:
df_toronto.shape

(180, 3)

<h2> Part 2: Getting the Geospatial coordinates for the neighborhoods </h2>

### Get Geo Coordinates of neighborhoods

In [228]:
df_coords = pd.read_csv("Geospatial_Coordinates.csv")
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [229]:
df_toronto = pd.merge(df_toronto,df_coords,left_on="PostalCode", right_on="Postal Code")
df_toronto = df_toronto.drop(columns=["Postal Code"])
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]",43.718518,-79.464763
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"[The Kingsway, Montgomery Road, Old Mill North]",43.653654,-79.506944
99,M4Y,Downtown Toronto,[Church and Wellesley],43.665860,-79.383160
100,M7Y,East Toronto,"[Business reply mail Processing Centre, South ...",43.662744,-79.321558
101,M8Y,Etobicoke,"[Old Mill South, King's Mill Park, Sunnylea, H...",43.636258,-79.498509


In [102]:
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    print(label)
    #label = folium.Popup(label, parse_html=True)

['Parkwoods'], North York
['Victoria Village'], North York
['Regent Park', 'Harbourfront'], Downtown Toronto
['Lawrence Manor', 'Lawrence Heights'], North York
["Queen's Park", 'Ontario Provincial Government'], Downtown Toronto
['Islington Avenue', 'Humber Valley Village'], Etobicoke
['Malvern', 'Rouge'], Scarborough
['Don Mills'], North York
['Parkview Hill', 'Woodbine Gardens'], East York
['Garden District', 'Ryerson'], Downtown Toronto
['Glencairn'], North York
['West Deane Park', 'Princess Gardens', 'Martin Grove', 'Islington', 'Cloverdale'], Etobicoke
['Rouge Hill', 'Port Union', 'Highland Creek'], Scarborough
['Don Mills'], North York
['Woodbine Heights'], East York
['St. James Town'], Downtown Toronto
['Humewood-Cedarvale'], York
['Eringate', 'Bloordale Gardens', 'Old Burnhamthorpe', 'Markland Wood'], Etobicoke
['Guildwood', 'Morningside', 'West Hill'], Scarborough
['The Beaches'], East Toronto
['Berczy Park'], Downtown Toronto
['Caledonia-Fairbanks'], York
['Woburn'], Scarborou

<h2> Part 3: Cluster and show data </h2>

<h3>Show the postal codes with thier according Borough and Neighborhood names on a map:</h3>

In [105]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

address = 'TORONTO, ON'
geolocator = Nominatim(user_agent="Coursera_Capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

#add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = folium.Popup(f"{neighborhood}, {borough}", parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto



<h3>Connect the Foursquare API</h3>

In [113]:
import requests # library to handle requests
CLIENT_ID = "" # your Foursquare ID
CLIENT_SECRET = ""# your Foursquare Secret
ACCESS_TOKEN = "" # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500 # define radius


### Test the API with just any location:

In [119]:
#choose the Neighborhood to explore.
ROW_NR = 1
neighborhood_latitude = df_toronto.loc[ROW_NR, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[ROW_NR, 'Longitude'] # neighborhood longitude value
neighborhood_name = df_toronto.loc[ROW_NR, 'Neighborhood'][0] # neighborhood name
print(f"Neighborhood of interest: {neighborhood_name} {neighborhood_latitude} {neighborhood_longitude}")

Neighborhood of interest: Victoria Village 43.725882299999995 -79.31557159999998


In [260]:
url_explore = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={neighborhood_latitude},{neighborhood_longitude}&radius={RADIUS}&limit={LIMIT}"
results = requests.get(url_explore).json()
len(results)

2

### Go though all Neighborhoods(coordinates) and get neaby venues and add all into one dataframe torono_venues

In [135]:
from pandas.io.json import json_normalize 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for names, lat, lng in zip(names, latitudes, longitudes):
        for name in names: #each postal code can have several names
            print(name)
                
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
                
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park
Harbourfront
Lawrence Manor
Lawrence Heights
Queen's Park
Ontario Provincial Government
Islington Avenue
Humber Valley Village
Malvern
Rouge
Don Mills
Parkview Hill
Woodbine Gardens
Garden District
Ryerson
Glencairn
West Deane Park
Princess Gardens
Martin Grove
Islington
Cloverdale
Rouge Hill
Port Union
Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
Bloordale Gardens
Old Burnhamthorpe
Markland Wood
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Wilson Heights
Downsview North
Thorncliffe Park
Richmond
Adelaide
King
Dufferin
Dovercourt Village
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Broadview North (Old East York)
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
Kennedy Park
Ionview
East Birchmount Park
Bayview Village
Downsvi

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Show venue count for each Neighborhood

In [173]:
toronto_venues.groupby('Neighborhood')["Venue"].count().sort_values(ascending=False)

Neighborhood
St. James Town     133
Adelaide           100
Design Exchange    100
Richmond           100
King               100
                  ... 
Rouge                1
Islington            1
Cloverdale           1
Humberlea            1
Silver Hills         1
Name: Venue, Length: 204, dtype: int64

### One-hot encode the venues for each neigborhood and then group by Neigborhood and get mean frequency of each Category to use this as vector for clustering

In [220]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
neighborhood = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neighborhood)

toronto_onehot_grouped_mean = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_onehot_grouped_mean


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,York Mills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,York University,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cluster using k-means into K clusters

In [256]:
from sklearn.cluster import KMeans

# set number of clusters
K = 5

toronto_grouped_clustering = toronto_onehot_grouped_mean.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=K, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 4, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       4, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       2, 1, 3, 3, 1, 1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 4, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 0, 1, 1, 4, 2, 1, 1, 2, 4, 1, 1, 3,
       1, 1, 4, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 4, 3, 1, 1, 1, 3, 1,
       1, 1, 4, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 3,
       1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 3, 1, 1,
       1, 1, 4, 4, 1, 1])

### Add the label to the onehot dataframe and join with the toronto dataframe

In [257]:
toronto_onehot_grouped_mean.drop("Cluster Labels",axis=1,inplace=True)
toronto_onehot_grouped_mean.insert(0,"Cluster Labels",kmeans.labels_)
toronto_onehot_grouped_mean.astype({'Cluster Labels': 'int32'})
toronto_onehot_grouped_mean.head()

,Cluster Labels,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,1,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [258]:
toronto_merged = df_toronto.copy()

#Neighborhood dtype is list. To join, convert using just the first value (cluster labels are the same anyway).
toronto_merged.rename(columns={"Neighborhood":"NeighborhoodLst"},inplace=True)
toronto_merged["Neighborhood"]=toronto_merged["Borough"]
for i in range(len(toronto_merged)):
    toronto_merged.iloc[i,toronto_merged.columns.get_loc('Neighborhood')]=toronto_merged.iloc[i].NeighborhoodLst[0]

#join dataframes and convert dtype of cluster labels
toronto_merged = toronto_merged.join(toronto_onehot_grouped_mean.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged = toronto_merged.astype({'Cluster Labels': 'int32'})

toronto_merged.head(10)

,PostalCode,Borough,NeighborhoodLst,Latitude,Longitude,Neighborhood,Cluster Labels,Accessories Store,Afghan Restaurant,Airport,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,North York,[Parkwoods],43.753259,-79.329656,Parkwoods,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000
1,M4A,North York,[Victoria Village],43.725882,-79.315572,Victoria Village,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]",43.654260,-79.360636,Regent Park,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.022727
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]",43.718518,-79.464763,Lawrence Manor,1,0.076923,0.0,0.0,...,0.0,0.0,0.0,0.00,0.076923,0.0,0.00,0.0,0.076923,0.000000
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]",43.662301,-79.389494,Queen's Park,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.030303
6,M1B,Scarborough,"[Malvern, Rouge]",43.806686,-79.194353,Malvern,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000
7,M3B,North York,[Don Mills],43.745906,-79.352188,Don Mills,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000
8,M4B,East York,"[Parkview Hill, Woodbine Gardens]",43.706397,-79.309937,Parkview Hill,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000
9,M5B,Downtown Toronto,"[Garden District, Ryerson]",43.657162,-79.378937,Garden District,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.01,0.010000,0.0,0.01,0.0,0.000000,0.000000
10,M6B,North York,[Glencairn],43.709577,-79.445073,Glencairn,1,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000


### Show toronto_merged dataframe on a map via Folium object and CircleMarker calls

In [259]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(K)
ys = [i + x + (i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, nbh, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(' Neighborhood:' +str(nbh) + ' Cluster:' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters